In [1]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
from IPython.display import Markdown, display


In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

deepseek_url = "https://api.deepseek.com"

openai = OpenAI()

OPENAI_MODEL = "gpt-5"


In [3]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()

message = f"""
Here is a report of the system information for my computer.
I want to run a C++ compiler to compile a single C++ file called main.cpp and then execute it in the simplest way possible.

Please tell me exactly and only what I should use for the compile_command and run_command in the format:
compile_command = # something here - to achieve the fastest possible runtime performance
run_command = # something here

System information:
{system_info}
"""

response = openai.chat.completions.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": message}])

In [4]:
import ast
import re

text = response.choices[0].message.content

compile_command = []
run_command = []

for line in text.split('\n'):
    if '=' in line:
        key, value = line.split('=', 1) 
        command_parts = value.strip().split()
        
        if key.strip() == 'compile_command':
            compile_command = command_parts
        elif key.strip() == 'run_command':
            run_command = command_parts


In [5]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [6]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [7]:
def write_output(cpp):
    with open("main.cpp", "w", encoding="utf-8") as f:
        f.write(cpp)

In [8]:
def port(client, model, python):
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)

In [9]:
def compile_and_run():
    subprocess.run(compile_command, check=True, text=True, capture_output=True)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)

In [10]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [11]:
def run_python(code):
    globals = {"__builtins__": __builtins__}
    exec(code, globals)
    
run_python(pi)    

Result: 3.141592656089
Execution Time: 12.083750 seconds


In [12]:
port(openai, OPENAI_MODEL, pi)
compile_and_run()

Result: 3.141592656091
Execution Time: 0.024438 seconds

Result: 3.141592656091
Execution Time: 0.014631 seconds

Result: 3.141592656091
Execution Time: 0.015588 seconds

